In [1]:
# This is Homework 1 for the 2025 cohort of the MLOps Zoomcamp.
# Due Date: May 19, 2025

In [2]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression

from sklearn.linear_model import Lasso
from sklearn.metrics import mean_squared_error
import pickle

In [3]:
!pip install pyarrow

In [4]:
pd.__version__

'1.4.2'

In [5]:
df_jan = pd.read_parquet('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-01.parquet')

In [6]:
df_jan.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
0,2,2023-01-01 00:32:10,2023-01-01 00:40:36,1.0,0.97,1.0,N,161,141,2,9.3,1.00,0.5,0.00,0.0,1.0,14.30,2.5,0.00
1,2,2023-01-01 00:55:08,2023-01-01 01:01:27,1.0,1.10,1.0,N,43,237,1,7.9,1.00,0.5,4.00,0.0,1.0,16.90,2.5,0.00
2,2,2023-01-01 00:25:04,2023-01-01 00:37:49,1.0,2.51,1.0,N,48,238,1,14.9,1.00,0.5,15.00,0.0,1.0,34.90,2.5,0.00
3,1,2023-01-01 00:03:48,2023-01-01 00:13:25,0.0,1.90,1.0,N,138,7,1,12.1,7.25,0.5,0.00,0.0,1.0,20.85,0.0,1.25
4,2,2023-01-01 00:10:29,2023-01-01 00:21:19,1.0,1.43,1.0,N,107,79,1,11.4,1.00,0.5,3.28,0.0,1.0,19.68,2.5,0.00


In [7]:
# Q1. Read the data for January. How many columns are there?

In [8]:
answer_1 = len(df_jan.columns)
print(answer_1)

19


In [9]:
# Q2. What's the standard deviation of the trips duration in January?

In [10]:
df_jan['duration'] = df_jan.tpep_dropoff_datetime - df_jan.tpep_pickup_datetime
df_jan.duration = df_jan.duration.apply(lambda td: td.total_seconds()/60)

In [11]:
df_jan.duration

0           8.433333
1           6.316667
2          12.750000
3           9.616667
4          10.833333
             ...    
3066761    13.983333
3066762    19.450000
3066763    24.516667
3066764    13.000000
3066765    14.400000
Name: duration, Length: 3066766, dtype: float64

In [12]:
answer_2 = df_jan.duration.std()
print(answer_2)

42.594351241920904


In [13]:
#Q3. What fraction of the records left after you dropped the outliers?

In [14]:
df_before = df_jan.copy()
df_after = df_jan[((df_jan.duration >= 1) & (df_jan.duration <= 60))]

answer_3 = (len(df_after) / len(df_before)) *100
print(answer_3)

98.1220282212598


In [15]:
# Q4. What's the dimensionality of this matrix (number of columns)?

In [16]:
# Turn dataframe ids intro strings
categorical = ['PULocationID', 'DOLocationID']
df_after[categorical] = df_jan[categorical].astype(str)

# Turn dataframe into list of dictionaries
train_dicts = df_after[categorical].to_dict(orient='records')

# Turns dictionary into a vector
dv = DictVectorizer()
X_train = dv.fit_transform(train_dicts)

answer_4 = X_train.shape[1]
print(answer_4)

/tmp/ipykernel_34037/750763771.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_after[categorical] = df_jan[categorical].astype(str)


515


In [17]:
# Q5. What's the RMSE on train?

In [18]:
target = 'duration'
y_train = df_after[target].values

lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_train)

mean_squared_error(y_train, y_pred, squared=False)
answer_5 = mean_squared_error(y_train, y_pred, squared=False)
print(answer_5)

7.6492610279057605


In [19]:
# Q6. What's the RMSE on validation?

In [20]:
df_val = pd.read_parquet('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-02.parquet')

df_val['duration'] = df_val.tpep_dropoff_datetime - df_val.tpep_pickup_datetime
df_val.duration = df_val.duration.apply(lambda td: td.total_seconds()/60)

df_val = df_val[((df_val.duration >= 1) & (df_val.duration <= 60))]

In [21]:
df_val[categorical] = df_val[categorical].astype(str)

In [22]:
val_dicts = df_val[categorical].to_dict(orient='records')

X_val = dv.transform(val_dicts)

In [23]:
y_val = df_val[target].values

y_pred = lr.predict(X_val)

mean_squared_error(y_val, y_pred, squared=False)
answer_6 = mean_squared_error(y_val, y_pred, squared=False)
print(answer_6)

7.81183265470218
